In [1]:
import os
import pandas as pd
import numpy as np
import ee

In [2]:
# 认证并初始化Google Earth Engine
ee.Authenticate()  # 只需要在第一次运行时执行
ee.Initialize(project= 'ee-zongrong-flood')

In [4]:
# Define the filtered dataset
dataset = ee.ImageCollection('MODIS/061/MOD13Q1').filter(ee.Filter.date('2023-01-01', '2023-12-31'))
ndvi_image = dataset.select('NDVI').mean()

# Calculate the min and max NDVI values for the filtered dataset
min_max = ndvi_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee.Geometry.BBox(-180, -90, 180, 90),
    scale=250,
    bestEffort=True
).getInfo()

min_value = min_max['NDVI_min']
max_value = min_max['NDVI_max']

print(f"Min value: {min_value}, Max value: {max_value}")

Min value: -1976, Max value: 9578


In [6]:
# Function to get NDVI value at given coordinates and normalize it
def get_ndvi_value(coords):
    point = ee.Geometry.Point(coords)
    ndvi_value = ndvi_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=250
    ).get('NDVI')
    
    value = ndvi_value.getInfo()
    if value is None or value < min_value or value > max_value:
        return 0
    
    # Normalize the NDVI value
    normalized_value = (value - min_value) / (max_value - min_value) * 10
    print(f"Coordinates: {coords}, Original value: {value}, Normalized value: {normalized_value}")  # Debugging line
    return normalized_value

# Specify the input and output directories
input_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/Sampling'
output_dir = 'C:/Users/lzr/SSCI/StreetviewLLM/groudtruth/ndvi'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each CSV file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        input_csv = os.path.join(input_dir, filename)
        df = pd.read_csv(input_csv)
        
        # Create a new column for NDVI values
        df['NDVI'] = df.apply(lambda row: get_ndvi_value([row['longitude'], row['latitude']]), axis=1)
        
        # Define the output filename
        output_csv = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_ndvi.csv")
        
        # Save the new dataframe to a CSV file
        df.to_csv(output_csv, index=False)
        
        print(f"Processed {input_csv} and saved with NDVI values as {output_csv}")

print("All CSV files processed.")

Coordinates: [114.276307, 22.2695341], Original value: 1504.6666666666667, Normalized value: 3.0125209162771913
Coordinates: [114.1023043902492, 22.37115357347453], Original value: 2615.6666666666665, Normalized value: 3.9740926663204656
Coordinates: [114.23624955805002, 22.287083445560008], Original value: 1754.625, Normalized value: 3.2288601350181754
Coordinates: [114.2983256, 22.2618995], Original value: 2738.875, Normalized value: 4.080729617448503
Coordinates: [114.18624955824998, 22.34541677866001], Original value: 4682.565217391304, Normalized value: 5.762995687546567
Coordinates: [114.1940713503955, 22.29561213573655], Original value: -453.75, Normalized value: 1.317509087761814
Coordinates: [114.1706041480453, 22.26166046887501], Original value: 5357.913043478261, Normalized value: 6.347509990893423
Coordinates: [114.25291622465, 22.30375011216001], Original value: 3705.2631578947367, Normalized value: 4.917139655439446
Coordinates: [114.21958289145, 22.32041677876001], Origi

Coordinates: [114.2479135727605, 22.35903170047959], Original value: 5639.391304347826, Normalized value: 6.591129742381709
Coordinates: [114.24458289135003, 22.32041677876001], Original value: 5019.913043478261, Normalized value: 6.054970610592228
Coordinates: [114.2027404, 22.3138097], Original value: 1460.75, Normalized value: 2.9745109918642894
Coordinates: [114.2868454941899, 22.24583502676082], Original value: 5462.5, Normalized value: 6.438030119439155
Coordinates: [114.18624955824998, 22.31208344546001], Original value: 3296.608695652174, Normalized value: 4.563448758570343
Coordinates: [114.2041615919109, 22.31957680366856], Original value: 1047.857142857143, Normalized value: 2.6171517594401443
Coordinates: [114.3082692, 22.3633873], Original value: 3125.714285714286, Normalized value: 4.415539454487006
Coordinates: [114.2240295, 22.36166], Original value: 6529.260869565217, Normalized value: 7.361312852315402
Coordinates: [114.275856, 22.2958202], Original value: 4043.476190

Coordinates: [114.0790097415447, 22.3602451276697], Original value: 663.5, Normalized value: 2.2844902198372856
Coordinates: [114.2285275, 22.2714013], Original value: 5521.739130434783, Normalized value: 6.489301653483455
Coordinates: [114.2603037001417, 22.34626232578898], Original value: 5925.782608695652, Normalized value: 6.839001738528347
Coordinates: [114.26958289125002, 22.29541677886001], Original value: 2346.764705882353, Normalized value: 3.7413577167062084
Coordinates: [114.10291622525, 22.34541677866001], Original value: 6757.363636363636, Normalized value: 7.558736053629596
Coordinates: [114.2941447628782, 22.3616944664604], Original value: 1962, Normalized value: 3.40834343084646
Coordinates: [114.11124955855, 22.370416778560006], Original value: 1989.9130434782608, Normalized value: 3.4325022013832966
Coordinates: [114.1997153221382, 22.36128378949011], Original value: 6680.391304347826, Normalized value: 7.492116413664381
Coordinates: [114.2603378, 22.2958908], Origina

Coordinates: [114.2220495763639, 22.34477299353125], Original value: 6242.95652173913, Normalized value: 7.11351611713617
Coordinates: [114.29458289115, 22.278750112260006], Original value: 4009.391304347826, Normalized value: 5.180362908384825
Coordinates: [114.14458289175002, 22.33708344536001], Original value: 2475.4347826086955, Normalized value: 3.852721812886183
Coordinates: [114.0881136, 22.3618831], Original value: 3042.818181818182, Normalized value: 4.343792783294255
Coordinates: [114.12791622515, 22.36208344526001], Original value: 3016.695652173913, Normalized value: 4.321183704495338
Coordinates: [114.3012422, 22.3210019], Original value: 200, Normalized value: 1.883330448329583
Coordinates: [114.2540134982365, 22.26213287085497], Original value: 5584.578947368421, Normalized value: 6.543689585743829
Coordinates: [114.15291622505003, 22.278750112260006], Original value: 2218.304347826087, Normalized value: 3.63017513227115
Coordinates: [114.24458289135003, 22.3287501120600

Coordinates: [114.3034106, 22.3039905], Original value: 5373.826086956522, Normalized value: 6.361282747928442
Coordinates: [114.1868668, 22.2867565], Original value: 1158.9545454545455, Normalized value: 2.7133066863896014
Coordinates: [114.2539469, 22.3294104], Original value: 6059.782608695652, Normalized value: 6.954978889298643
Coordinates: [114.268924237331, 22.3141902527644], Original value: 4838.826086956522, Normalized value: 5.89823964597241
Coordinates: [114.2274961511648, 22.25397835546077], Original value: 6181.478260869565, Normalized value: 7.060306613181205
Coordinates: [114.0868046, 22.3705706], Original value: 6810.666666666667, Normalized value: 7.604869886330853
Coordinates: [114.1184305, 22.3355833], Original value: 209.6, Normalized value: 1.891639259131037
Coordinates: [114.13624955845, 22.287083445560008], Original value: 1957.8095238095239, Normalized value: 3.4047165689886825
Coordinates: [114.17791622495002, 22.287083445560008], Original value: -14.625, Norma

...

Coordinates: [139.47791612375, 35.45375005956001], Original value: 3509.782608695652, Normalized value: 4.747951020162413
Coordinates: [139.50291612365004, 35.64541672546001], Original value: 2999.695652173913, Normalized value: 4.306470185367763
Coordinates: [139.57791612335, 35.58708339236001], Original value: 2763.304347826087, Normalized value: 4.101873245478698
Coordinates: [139.72791612275, 35.59541672566001], Original value: 1992.4347826086957, Normalized value: 3.434684769438027
Coordinates: [140.10291612125002, 35.50375005936001], Original value: 2974.1739130434785, Normalized value: 4.2843810914345495
Coordinates: [139.7034057982496, 35.55411392345233], Original value: 2507, Normalized value: 3.880041544054007
Coordinates: [139.4446702528568, 35.89607593054409], Original value: 3912.478260869565, Normalized value: 5.096484560212536
Coordinates: [139.51958279025, 35.62041672556001], Original value: 4065.0434782608695, Normalized value: 5.228529927523688
Coordinates: [139.89458

Coordinates: [139.66124945635, 35.88708339116001], Original value: 2640.1739130434785, Normalized value: 3.9953037156339604
Coordinates: [139.81958278905, 35.77041672496001], Original value: 2046.7826086956522, Normalized value: 3.481722874065823
Coordinates: [139.54458279015, 35.67041672536001], Original value: 5311, Normalized value: 6.306906698978708
Coordinates: [140.11958278785, 35.52875005926001], Original value: 3486.3478260869565, Normalized value: 4.727668189446907
Coordinates: [139.337255, 35.886842], Original value: 6000.130434782609, Normalized value: 6.903349865659173
Coordinates: [139.3036586194734, 35.75439333799493], Original value: 6775.652173913043, Normalized value: 7.574564803455984
Coordinates: [139.770537916088, 35.59424132477426], Original value: 1125.608695652174, Normalized value: 2.6844458158665176
Coordinates: [139.44458279055, 35.53708339256001], Original value: 2973.3478260869565, Normalized value: 4.283666112244207
Coordinates: [140.06958278805, 35.8954167

Coordinates: [139.96124945514998, 35.853750057960006], Original value: 2272.2608695652175, Normalized value: 3.676874562545627
Coordinates: [139.78624945585, 35.72041672516001], Original value: 1264.2608695652175, Normalized value: 2.8044494283929526
Coordinates: [139.35291612425, 35.71208339186001], Original value: 2742.8260869565215, Normalized value: 4.0841492876549434
Coordinates: [139.72791612275, 35.80375005816001], Original value: 1494.1304347826087, Normalized value: 3.003401795726682
Coordinates: [140.2477291748352, 35.81368122517575], Original value: 4711.772727272727, Normalized value: 5.788274820211812
Coordinates: [140.26958278725, 35.66208339206001], Original value: 4848.869565217391, Normalized value: 5.906932287707626
Coordinates: [139.76124945595, 35.54541672586001], Original value: 1700.2857142857142, Normalized value: 3.1818294220925343
Coordinates: [140.21958278745, 35.87875005786001], Original value: 4582.217391304348, Normalized value: 5.676144531161804
Coordinate

Coordinates: [139.81124945575, 35.77875005826001], Original value: 1632.6521739130435, Normalized value: 3.123292516802011
Coordinates: [140.22791612075002, 35.77875005826001], Original value: 6407.739130434783, Normalized value: 7.256135650367651
Coordinates: [139.45291612385, 35.57875005906001], Original value: 4220.086956521739, Normalized value: 5.362720232405867
Coordinates: [140.11124945455003, 35.82041672476001], Original value: 5674, Normalized value: 6.62108360740869
Coordinates: [140.2059368414895, 35.52024416861443], Original value: 7502.521739130435, Normalized value: 8.203671229989991
Coordinates: [140.24458278735, 35.88708339116001], Original value: 4170.521739130435, Normalized value: 5.3198214809853175
Coordinates: [139.44458279055, 35.48708339276001], Original value: 2321.8695652173915, Normalized value: 3.719810944449881
Coordinates: [139.45291612385, 35.74541672506001], Original value: 2767.913043478261, Normalized value: 4.1058620767511345
Coordinates: [140.12942859

Coordinates: [140.21958278745, 35.62041672556001], Original value: 5944.652173913043, Normalized value: 6.855333368455118
Coordinates: [139.48624945705, 35.47875005946001], Original value: 3135.8695652173915, Normalized value: 4.424328860323171
Coordinates: [139.76958278925002, 35.87875005786001], Original value: 2589.2608695652175, Normalized value: 3.951238419218641
Coordinates: [139.71124945615003, 35.52041672596001], Original value: 1628.0869565217392, Normalized value: 3.119341316013276
Coordinates: [139.57791612335, 35.62875005886001], Original value: 2459.2608695652175, Normalized value: 3.8387232729489513
Coordinates: [140.28624945385002, 35.67041672536001], Original value: 4881.739130434783, Normalized value: 5.935380933386519
Coordinates: [140.19458278755002, 35.63708339216001], Original value: 6929.521739130435, Normalized value: 7.707739085278203
Coordinates: [139.55291612345002, 35.77875005826001], Original value: 4428.347826086957, Normalized value: 5.542970249339585
Coor